# Building user-based recommendation model for Amazon

## IMPORTS

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
sns.set(style="darkgrid")
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action="ignore")
import re
import scipy as sp

## Reading the CSV

In [2]:
Movies_df = pd.read_csv("Amazon - Movies and TV Ratings.csv")
Movies_df.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
Movies_df.shape

(4848, 207)

In [4]:
Movies_df.size

1003536

In [5]:
Movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4848 entries, 0 to 4847
Columns: 207 entries, user_id to Movie206
dtypes: float64(206), object(1)
memory usage: 7.7+ MB


In [6]:
Movies_df.isna().sum()

user_id        0
Movie1      4847
Movie2      4847
Movie3      4847
Movie4      4846
            ... 
Movie202    4842
Movie203    4847
Movie204    4840
Movie205    4813
Movie206    4835
Length: 207, dtype: int64

### Cleaning the data

In [7]:
Movies_df.describe()

,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,Movie10,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
count,1.0,1.0,1.0,2.0,29.000000,1.0,1.0,1.0,1.0,1.0,...,5.000000,2.0,1.0,8.000000,3.000000,6.000000,1.0,8.000000,35.000000,13.000000
mean,5.0,5.0,2.0,5.0,4.103448,4.0,5.0,5.0,5.0,5.0,...,3.800000,5.0,5.0,4.625000,4.333333,4.333333,3.0,4.375000,4.628571,4.923077
std,NaN,NaN,NaN,0.0,1.496301,NaN,NaN,NaN,NaN,NaN,...,1.643168,0.0,NaN,0.517549,1.154701,1.632993,NaN,1.407886,0.910259,0.277350
min,5.0,5.0,2.0,5.0,1.000000,4.0,5.0,5.0,5.0,5.0,...,1.000000,5.0,5.0,4.000000,3.000000,1.000000,3.0,1.000000,1.000000,4.000000
25%,5.0,5.0,2.0,5.0,4.000000,4.0,5.0,5.0,5.0,5.0,...,4.000000,5.0,5.0,4.000000,4.000000,5.000000,3.0,4.750000,5.000000,5.000000
50%,5.0,5.0,2.0,5.0,5.000000,4.0,5.0,5.0,5.0,5.0,...,4.000000,5.0,5.0,5.000000,5.000000,5.000000,3.0,5.000000,5.000000,5.000000
75%,5.0,5.0,2.0,5.0,5.000000,4.0,5.0,5.0,5.0,5.0,...,5.000000,5.0,5.0,5.000000,5.000000,5.000000,3.0,5.000000,5.000000,5.000000
max,5.0,5.0,2.0,5.0,5.000000,4.0,5.0,5.0,5.0,5.0,...,5.000000,5.0,5.0,5.000000,5.000000,5.000000,3.0,5.000000,5.000000,5.000000


In [8]:
# Copy1
Movies_df1 = Movies_df.fillna(value=0)
Movies_df1.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AH3QC2PC1VTGP,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A3LKP6WPMP9UKX,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AVIY68KEPQ5ZD,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A1CV1WROP5KTTW,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
Movies_df1.describe()

,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,Movie10,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
count,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,...,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000
mean,0.001031,0.001031,0.000413,0.002063,0.024546,0.000825,0.001031,0.001031,0.001031,0.001031,...,0.003919,0.002063,0.001031,0.007632,0.002682,0.005363,0.000619,0.007219,0.033416,0.013201
std,0.071811,0.071811,0.028724,0.101545,0.336268,0.057448,0.071811,0.071811,0.071811,0.071811,...,0.130800,0.101545,0.071811,0.188769,0.110296,0.161142,0.043086,0.185478,0.399243,0.254991
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,5.000000,2.000000,5.000000,5.000000,4.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,5.000000,5.000000,5.000000


In [10]:
# Copy2
Movies_df2 = Movies_df1.drop(columns='user_id')
Movies_df2.head()

,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,Movie10,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
Movies_df2.describe()

,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,Movie10,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
count,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,...,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000,4848.000000
mean,0.001031,0.001031,0.000413,0.002063,0.024546,0.000825,0.001031,0.001031,0.001031,0.001031,...,0.003919,0.002063,0.001031,0.007632,0.002682,0.005363,0.000619,0.007219,0.033416,0.013201
std,0.071811,0.071811,0.028724,0.101545,0.336268,0.057448,0.071811,0.071811,0.071811,0.071811,...,0.130800,0.101545,0.071811,0.188769,0.110296,0.161142,0.043086,0.185478,0.399243,0.254991
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,5.000000,2.000000,5.000000,5.000000,4.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,5.000000,5.000000,5.000000


# Analysis Task

## Exploratory Data Analysis:
### Which movies have maximum views/ratings?

In [12]:
# VIEWS
Movies_df.describe().T["count"].sort_values(ascending = False)[0:6]

Movie127    2313.0
Movie140     578.0
Movie16      320.0
Movie103     272.0
Movie29      243.0
Movie91      128.0
Name: count, dtype: float64

In [13]:
Amazon_max_views = Movies_df2.sum()
Amazon_max_views

Movie1        5.0
Movie2        5.0
Movie3        2.0
Movie4       10.0
Movie5      119.0
            ...  
Movie202     26.0
Movie203      3.0
Movie204     35.0
Movie205    162.0
Movie206     64.0
Length: 206, dtype: float64

In [14]:
max(Amazon_max_views)

9511.0

In [15]:
max_views= Amazon_max_views.argmax()
max_views

126

In [16]:
Amazon_max_views['Movie127']

9511.0

In [17]:
Amazon_max_views.head()

Movie1      5.0
Movie2      5.0
Movie3      2.0
Movie4     10.0
Movie5    119.0
dtype: float64

In [18]:
Amazon_max_views.tail()

Movie202     26.0
Movie203      3.0
Movie204     35.0
Movie205    162.0
Movie206     64.0
dtype: float64

### What is the average rating for each movie?

In [19]:
Amazon_max_views.mean()

106.44660194174757

In [20]:
Amazon_df = pd.DataFrame(Amazon_max_views)
Amazon_df.head()

,0
Movie1,5.0
Movie2,5.0
Movie3,2.0
Movie4,10.0
Movie5,119.0


In [21]:
Amazon_df.columns=['rating']

In [22]:
Amazon_df.index

Index(['Movie1', 'Movie2', 'Movie3', 'Movie4', 'Movie5', 'Movie6', 'Movie7',
       'Movie8', 'Movie9', 'Movie10',
       ...
       'Movie197', 'Movie198', 'Movie199', 'Movie200', 'Movie201', 'Movie202',
       'Movie203', 'Movie204', 'Movie205', 'Movie206'],
      dtype='object', length=206)

In [23]:
Amazon_df.head(6)

,rating
Movie1,5.0
Movie2,5.0
Movie3,2.0
Movie4,10.0
Movie5,119.0
Movie6,4.0


### Define the top 5 movies with the maximum ratings.

In [24]:
Amazon_df.nlargest(5,'rating')

,rating
Movie127,9511.0
Movie140,2794.0
Movie16,1446.0
Movie103,1241.0
Movie29,1168.0


### Define the top 5 movies with the least audience.

In [25]:
Amazon_df.nsmallest(5,'rating')

,rating
Movie45,1.0
Movie58,1.0
Movie60,1.0
Movie67,1.0
Movie69,1.0


## Recommendation Model: Some of the movies hadn’t been watched and therefore, are not rated by the users. Netflix would like to take this as an opportunity and build a machine learning recommendation algorithm which provides the ratings for each of the users.

### Imports

In [26]:
# Suprise package is used for recommender systems.
# http://surpriselib.com/
import surprise
from surprise import Reader
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [27]:
# Melting the data From Movies_df original dataset
amazon_pd = pd.DataFrame(Movies_df)
melt_df=amazon_pd.melt(id_vars= Movies_df.columns[0],value_vars=Movies_df.columns[1:],var_name='Movie',value_name='rating')
melt_df

,user_id,Movie,rating
0,A3R5OBKS7OM2IR,Movie1,5.0
1,AH3QC2PC1VTGP,Movie1,NaN
2,A3LKP6WPMP9UKX,Movie1,NaN
3,AVIY68KEPQ5ZD,Movie1,NaN
4,A1CV1WROP5KTTW,Movie1,NaN
...,...,...,...
998683,A1IMQ9WMFYKWH5,Movie206,5.0
998684,A1KLIKPUF5E88I,Movie206,5.0
998685,A5HG6WFZLO10D,Movie206,5.0
998686,A3UU690TWXCG1X,Movie206,5.0


In [28]:
melt_df.shape

(998688, 3)

In [29]:
melt_filtered = melt_df.fillna(0)
melt_filtered.shape

(998688, 3)

In [30]:
melt_filtered.head()

,user_id,Movie,rating
0,A3R5OBKS7OM2IR,Movie1,5.0
1,AH3QC2PC1VTGP,Movie1,0.0
2,A3LKP6WPMP9UKX,Movie1,0.0
3,AVIY68KEPQ5ZD,Movie1,0.0
4,A1CV1WROP5KTTW,Movie1,0.0


### - Divide the data into training and test data

In [31]:
reader = Reader(rating_scale=(-1,10))
data = Dataset.load_from_df(melt_df.fillna(0), reader=reader)

In [32]:
trainset, testset = train_test_split(data, test_size=0.25)

### - Build a recommendation model on training data

In [33]:
#Building a model
algo = SVD()
algo.fit(trainset)

### - Make predictions on the test data

In [34]:
cross_validate(algo,data,measures=['RMSE','MAE'],cv=3,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.2746  0.2853  0.2868  0.2822  0.0054  
MAE (testset)     0.0423  0.0435  0.0434  0.0431  0.0006  
Fit time          47.31   46.17   46.71   46.73   0.47    
Test time         4.37    4.24    4.25    4.29    0.06    


{'test_rmse': array([0.27463608, 0.28525915, 0.28680457]),
 'test_mae': array([0.04227907, 0.04350976, 0.0434109 ]),
 'fit_time': (47.31264281272888, 46.166625022888184, 46.70634365081787),
 'test_time': (4.374154090881348, 4.24066948890686, 4.2457802295684814)}

In [35]:
user_id='A1CV1WROP5KTTW'
Movie='Movie6'
rating='5'
algo.predict(user_id,Movie,r_ui=rating)
print(cross_validate(algo,data,measures=['RMSE','MAE'],cv=3,verbose=True))

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.2876  0.2810  0.2783  0.2823  0.0039  
MAE (testset)     0.0437  0.0425  0.0425  0.0429  0.0006  
Fit time          46.20   52.79   49.06   49.35   2.69    
Test time         3.74    4.50    4.20    4.15    0.31    
{'test_rmse': array([0.28757686, 0.28098669, 0.27834537]), 'test_mae': array([0.04373791, 0.04250523, 0.04254034]), 'fit_time': (46.204550981521606, 52.78693461418152, 49.062886476516724), 'test_time': (3.7419729232788086, 4.500999450683594, 4.195788621902466)}
